<a href="https://colab.research.google.com/github/ArthurHSM/BigData_Analyticis_R_Azure/blob/main/1_EDA_Modelagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
# Instalar o Java Development Kit (JDK) - necessário para o Spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Instalar o PySpark
!pip install pyspark --quiet
!pip install findspark --quiet # Para ajudar o Colab a encontrar o Spark
!pip install imbalanced-learn --quiet # Manter esta para tratamento de desbalanceamento em Pandas/Scikit-learn

In [ ]:
import findspark
# Configura o local de instalação do Spark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *

# Cria a sessão Spark
spark = SparkSession.builder\
    .appName("ITIncidentClassifier")\
    .master("local[*]")\
    .getOrCreate()

print("Spark Session iniciada com sucesso!")

# Ferramentas Essenciais
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Para machine learning (vamos usar o scikit-learn para a modelagem)
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE # Para tratar o desbalanceamento em um subconjunto

Spark Session iniciada com sucesso!


# Leitura do dataset

In [4]:
# Conexão com o Google Drive (se seu arquivo estiver lá)
from google.colab import drive
drive.mount('/content/drive')

# Exemplo de carregamento de arquivo CSV usando PySpark
# Lembre-se de ajustar o caminho!
data_path = '/content/drive/MyDrive/01-FIAP/TechChallenge_03/alerts_dataset.csv'

df_spark = spark.read.csv(
    path=data_path,
    header=True,
    inferSchema=True
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
df_spark

alert_id,source,environment,severity,metric_name,description,ci,created_at,updated_at,is_incident
9d050826-4e3a-459...,CloudWatch,prod,clear,service_unavailable,High service_unav...,router-16,2025-07-16 00:08:32,2025-07-16 13:00:32,0
4de72c92-2e87-4d3...,Dynatrace,prod,major,db_connections,Db_connections ab...,server-13,2025-09-29 10:12:37,2025-09-30 08:05:37,1
73a4448a-43f7-4e1...,ThousandEyes,hml,major,cpu_usage,Cpu usage spike o...,db-3,2025-09-23 01:14:12,2025-09-23 01:42:12,0
b2c6910c-b673-4c6...,Dynatrace,prod,ok,ping_failed,Multiple ping fai...,server-69,2025-07-07 19:42:15,2025-07-08 10:44:15,0
f27a8c9b-d405-42b...,CloudWatch,hml,critical,http_response_time,Alert: http respo...,switch-16,2025-09-17 17:38:51,2025-09-18 03:32:51,0
252923c6-ebb9-450...,Zabbix,prod,ok,queue_length,Multiple queue le...,app-53,2025-09-28 18:01:18,2025-09-28 23:35:18,1
d84c7a25-2ede-4e4...,Zabbix,hml,critical,http_response_time,Http response tim...,server-37,2025-08-31 19:09:06,2025-09-01 03:48:06,0
85522b8f-0f7a-4b4...,CloudWatch,prod,ok,queue_length,Alert: queue leng...,server-55,2025-08-08 14:51:09,2025-08-08 18:31:09,0
91ad3583-d6ff-412...,Zabbix,prod,ok,response_time,Response_time abo...,db-39,2025-07-24 14:40:42,2025-07-24 22:14:42,0
0b8de405-5e96-4f3...,CloudWatch,hml,minor,ping_failed,Ping failed criti...,db-8,2025-08-04 16:30:32,2025-08-04 22:05:32,0


# EDA - Exploratory Data Analysis

In [17]:


# Considerando que se trata de um problema de classificação, é importante saber a distribuição da variável alvo.
# Quantidades maiores de uma classe em relação a outra podem enviezar o modelo.
# Usar modelos como Random Forest ou XGBoost podem encaixar bem na distribuição abaixo. Lembrar de usar o argumento de class_weights no treinamento.
df_spark.groupBy('is_incident').count().show()

+-----------+-----+
|is_incident|count|
+-----------+-----+
|          1|14200|
|          0|35800|
+-----------+-----+



In [26]:
# Analisando a proporção de incidentes por tipo de "severity"

df_severity_analysis = (
    df_spark
    .groupBy('severity')
    .agg(
        F.countDistinct('alert_id').alias('total_alerts'),
        F.sum(F.col('is_incident')).alias('total_incidents')
    )
    .withColumn('perc_incidents', F.round((F.col("total_incidents") / F.col("total_alerts"))*100, 2))
    .orderBy(F.col('perc_incidents').desc())
)

df_severity_analysis.show()

# Analisando a proporção de incidentes por tipo de "source"

df_source_analysis = (
    df_spark
    .groupBy('source')
    .agg(
        F.countDistinct('alert_id').alias('total_alerts'),
        F.sum(F.col('is_incident')).alias('total_incidents')
    )
    .withColumn('perc_incidents', F.round((F.col("total_incidents") / F.col("total_alerts"))*100, 2))
    .orderBy(F.col('perc_incidents').desc())
)

df_source_analysis.show()

# severidades 'major' e 'critical' parecem ter alta relação positiva com incidentes. Enquanto a 'source' parece não ser uma boa variável preditora (pelo menos quando analisada isoladamente)

+--------+------------+---------------+--------------+
|severity|total_alerts|total_incidents|perc_incidents|
+--------+------------+---------------+--------------+
|   major|        5082|           3067|         60.35|
|critical|        3834|           2303|         60.07|
| warning|       12466|           4757|         38.16|
|   minor|        7504|           1695|         22.59|
|      ok|       17679|           2070|         11.71|
|   clear|        3435|            308|          8.97|
+--------+------------+---------------+--------------+

+------------+------------+---------------+--------------+
|      source|total_alerts|total_incidents|perc_incidents|
+------------+------------+---------------+--------------+
|      Zabbix|        9994|           2880|         28.82|
|  CloudWatch|       10041|           2891|         28.79|
|   Dynatrace|       24969|           7124|         28.53|
|ThousandEyes|        4996|           1305|         26.12|
+------------+------------+---------

In [39]:
# Supondo que 'created_at' seja uma string formatada (Ex: 2025-07-16 08:00:32)
# Se não for do tipo TIMESTAMP, converta primeiro.

# Ação 3: Contagem de valores únicos em 'ci'
ci_count = df_spark.select(F.countDistinct("ci")).collect()[0][0]
print(f"Número de CIs (Ativos) únicos: {ci_count}")


# Ação 4: Sazonalidade (Hora e Dia da Semana)
df_time_analysis = df_spark.withColumn(
    "hour", F.hour(F.col("created_at"))
).withColumn(
    "day_of_week", F.dayofweek(F.col("created_at")) # Domingo=1, Sábado=7
)

# Calcula a taxa de incidente por hora
df_incident_rate_by_hour = df_time_analysis.groupBy("hour").agg(
    F.countDistinct(F.col("alert_id")).alias("total_alerts"),
    F.sum(F.col('is_incident')).alias('total_incidents'),
).withColumn('perc_incidents_by_hour', F.round((F.col("total_incidents") / F.col("total_alerts"))*100, 2)).sort(F.col("hour"))

df_incident_rate_by_hour.show(24)

# Calcula a taxa de incidente por dia da semana
df_incident_rate_by_day_of_week = df_time_analysis.groupBy("day_of_week").agg(
    F.countDistinct(F.col("alert_id")).alias("total_alerts"),
    F.sum(F.col('is_incident')).alias('total_incidents'),
).withColumn('perc_incidents_by_day_of_week', F.round((F.col("total_incidents") / F.col("total_alerts"))*100, 2)).sort(F.col("day_of_week"))

df_incident_rate_by_day_of_week.show()

Número de CIs (Ativos) únicos: 225
+----+------------+---------------+----------------------+
|hour|total_alerts|total_incidents|perc_incidents_by_hour|
+----+------------+---------------+----------------------+
|   0|        2079|            588|                 28.28|
|   1|        2169|            601|                 27.71|
|   2|        2106|            624|                 29.63|
|   3|        1998|            582|                 29.13|
|   4|        2089|            588|                 28.15|
|   5|        2072|            568|                 27.41|
|   6|        2032|            572|                 28.15|
|   7|        2066|            587|                 28.41|
|   8|        2090|            613|                 29.33|
|   9|        2148|            565|                  26.3|
|  10|        2062|            551|                 26.72|
|  11|        2116|            616|                 29.11|
|  12|        2080|            614|                 29.52|
|  13|        2038|  